# Quantising a MobileNetV2 with the PACT algorithm

In this notebook, we show how to create an integerised MobileNetV2 using the QuantLib package.
Differently from the MobileNetV1 architecture and similarly to ResNets, MobileNetV2 has a branching topology, i.e., it uses blocks modelling a function as the sum of the identity (*skip branch*) and a non-identity (*residual branch*) function.


In [1]:
from __future__ import annotations

from typing import NamedTuple, List, Union, Optional, Type


## Part 1: creating and evaluating a floating-point network

### Step 1: check the computing infrastructure

Depending on the hardware we have, we will make different choices about the training and testing processes.


In [2]:
import multiprocessing
import torch

n_cpus = multiprocessing.cpu_count()
n_gpus = torch.cuda.device_count() if torch.cuda.is_available() else 0
print(f"Available system CPU(s): {n_cpus}.")
print(f"Available system GPU(s): {n_gpus}.")

device = torch.device(torch.cuda.current_device()) if (n_gpus > 0) else torch.device('cpu')


Available system CPU(s): 20.
Available system GPU(s): 2.


### Step 2: create PyTorch data loaders

Each supervised learning problem partitions data points into three categories:
* **training** points: they are available at training time, and are consumed by the learning algorithm to update the model's parameters;
* **validation** points: they are available at training time, they are not used to update the parameters, but they can be used to measure the performance of the model throughout the learning process;
* **test** points: they are not available at training time, but are available only once the model is deployed in production.


In [3]:
from enum import Enum, auto, unique


# each supervised learning problem defines a training/validation/test partition of its data points
@unique
class DataSetPartition(Enum):
    TRAINING   = auto()
    VALIDATION = auto()
    TEST       = auto()

    @classmethod
    def canonicalise(cls, partition_spec: DataSetPartitionSpec) -> DataSetPartition:

        # validate specification type
        if not isinstance(partition_spec, (DataSetPartition, str,)):
            raise TypeError

        if isinstance(partition_spec, DataSetPartition):
            partition = partition_spec
        
        else:  # `isinstance(partition, str)`

            partition_spec = partition_spec.upper()
            if partition_spec in {'TRAINING', 'TRAIN'}:
                partition = cls['TRAINING']
            elif partition_spec in {'VALIDATION', 'VALID'}:
                partition = cls['VALIDATION']
            elif partition_spec in {'TEST', 'TESTING'}:
                partition = cls['TEST']
            else:
                raise ValueError
        
        return partition

    
# define the ways in which a user can specify data set partitions
DataSetPartitionSpec = Union[DataSetPartition, str]


PyTorch represents data points using the `torch.Tensor` data structure.
PyTorch uses a specific pipeline to transform files stored on disk into mini-batches of `torch.Tensor`s.
This pipeline consists of three stages.
* Define a function to transform each data point stored on disk into a corresponding collection of `torch.Tensor`s; for instance, a labelled image file can be mapped to a pair consisting of an input tensor and a label tensor. These functions are called *transforms*, and might be composed to describe complex pre-processing transformations (see `torchvision.transforms.Compose`). Since *transforms* can be applied individually to each data point, `torch.utils.data.Dataset` objects apply a *map pattern* to load the files on disk into a Python list.
* Define a `torch.utils.data.Sampler` that can sample and return mini-batches from a given list of integers. This batching can be performed with or without permuting the list; if the list is permuted, it can be permuted once (without repetition), or in-between each sampling (with repetition).
* Define a `torch.utils.data.DataLoader` specifying the size of the mini-batches and how many worker threads should be used to load data point files from disk. The `DataLoader` queries the `Sampler` for a list of indices, distributes these indices to the worker threads, each thread returns the `Tensor` data points, the `DataLoader` collates them into a mini-batch of `Tensor`s, and returns these mini-batches.


In [4]:
from collections import OrderedDict
import os
import torchvision


class ImageNetDataLoaderFactory(object):
    
    def __init__(self, path_data: str):
        
        if not os.path.isdir(path_data):
            raise FileNotFounderror  # missing ImageNet data folder

        super(ImageNetDataLoaderFactory, self).__init__()

        self._partition_to_subfolder = OrderedDict([
            (DataSetPartition['TRAINING'], os.path.join(path_data, 'train')),
            (DataSetPartition['VALIDATION'], os.path.join(path_data, 'val')),
        ])

        
    def get_dataset(self,
                    partition_spec: DataSetPartitionSpec,
                    transform:      torchvision.transforms.Compose) -> torch.utils.data.Dataset:
        partition = DataSetPartition.canonicalise(partition_spec)
        return torchvision.datasets.ImageFolder(self._partition_to_subfolder[partition], transform)
    
    @staticmethod
    def get_sampler(partition_spec: DataSetPartitionSpec,
                    dataset: torch.utils.data.Dataset) -> torch.utils.data.Sampler:
        partition = DataSetPartition.canonicalise(partition_spec)
        return torch.utils.data.RandomSampler(dataset) if (partition is DataSetPartition['TRAINING']) else torch.utils.data.SequentialSampler(dataset)
    
    def get_dataloader(self,
                       partition_spec: DataSetPartitionSpec,
                       transform:      torchvision.transforms.Compose,
                       batch_size:     int,
                       num_workers:    int = 1) -> torch.utils.data.DataLoader:

        partition = DataSetPartition.canonicalise(partition_spec)

        dataset = self.get_dataset(partition, transform)
        sampler = ImageNetDataLoaderFactory.get_sampler(partition, dataset)
        loader = torch.utils.data.DataLoader(dataset=dataset,
                                             sampler=sampler,
                                             batch_size=batch_size,
                                             num_workers=num_workers)

        return loader


Now, we define ImageNet-specific transforms.

In [5]:
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms import Resize, CenterCrop
from torchvision.transforms import ToTensor, Normalize, Lambda
from torchvision.transforms import Compose


ImageNetStats = \
    {
        'normalise':
            {
                'mean': (0.485, 0.456, 0.406),
                'std':  (0.229, 0.224, 0.225)
            },
        'quantise':
            {
                'min':   -2.1179039478,  # computed on the normalised images of the validation partition
                'max':   2.6400001049,   # computed on the normalised images of the validation partition
                'scale': 0.020625000819563866
            }
    }


class ImageNetNormalise(Normalize):
    def __init__(self):
        super(ImageNetNormalise, self).__init__(**ImageNetStats['normalise'])
        

class ImageNetIntegerise(Lambda):
    def __init__(self):
        INT8_MIN = -2**(8-1)
        INT8_MAX = 2**(8-1) - 1
        image_scale = ImageNetStats['quantise']['scale']
        super(ImageNetIntegerise, self).__init__(lambda x: torch.clip((x / image_scale).floor(), INT8_MIN, INT8_MAX))


class ImageNetTransform(Compose):

    def __init__(self, partition_spec: DataSetPartitionSpec, image_size: int = 224, integerise: bool = False):

        # validate arguments
        RESIZE_SIZE = 256
        if not (image_size <= RESIZE_SIZE):
            raise ValueError  # otherwise, we can not crop the resized image to the desired size

        partition = DataSetPartition.canonicalise(partition_spec)

        if partition is DataSetPartition['TRAINING']:
            transforms = [RandomHorizontalFlip(),
                          RandomResizedCrop(image_size)]
        else:
            transforms = [Resize(RESIZE_SIZE),
                          CenterCrop(image_size)]

        transforms += [ToTensor(),
                       ImageNetNormalise(),
                       ImageNetIntegerise()]
        
        if not integerise:
            transforms += [Lambda(lambda x: x * ImageNetStats['quantise']['scale'])]  # return a fake-quantised `Tensor`

        super(ImageNetTransform, self).__init__(transforms)


We are now ready to create our training and validation `DataLoader`s.

In [6]:
# create the `DataLoader` factory
path_data = os.path.join(os.curdir, 'data')
loader_factory = ImageNetDataLoaderFactory(path_data)

per_gpu_batch_size = 48
batch_size = max(1, n_gpus) * per_gpu_batch_size  # `nn.DataParallel` will revert this dispatching `per_gpu_batch_size` items to each GPU

# create the training `DataLoader`
train_transform = ImageNetTransform('valid')  # no augmentation  # TODO: this is Georg's finding, so I can not justify the rationale
train_loader = loader_factory.get_dataloader('train', train_transform, batch_size, num_workers=n_cpus)

# create the validation `DataLoader`
valid_transform = ImageNetTransform('valid')
valid_loader = loader_factory.get_dataloader('valid', valid_transform, batch_size, num_workers=n_cpus)


### Step 3: create a floating-point PyTorch network

PyTorch represents deep neural networks as `torch.nn.Module`s.
Since deep neural networks can be (and are often) modelled as function compositions, `nn.Module`s can be composed to create complex functions, i.e., complex networks.

MobileNetV1 is a sequential composition of `nn.Module`s, therefore it is a relatively simple network to quantise.


In [7]:
import torch.nn as nn


CONFIGS = OrderedDict([
    ('STANDARD', [
        # t,  c, n, s
        [1,  16, 1, 1],
        [6,  24, 2, 2],
        [6,  32, 3, 2],
        [6,  64, 4, 2],
        [6,  96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1]
    ]),
])


ACTIVATIONS = ('relu', 'relu6',)


class Conv2dBN2dActivation(nn.Sequential):

    def __init__(self,
                 in_channels:        int,
                 out_channels:       int,
                 kernel_size:        int = 3,
                 stride:             int = 1,
                 groups:             int = 1,
                 activation_class:   Type[nn.Module] = nn.ReLU6):

        modules = []
        modules += [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=(kernel_size - 1) // 2, groups=groups, bias=False)]
        modules += [nn.BatchNorm2d(out_channels)]
        modules += [activation_class(inplace=True)]  # we assume that the `activation_class` is "inplaceable"

        super(Conv2dBN2dActivation, self).__init__(*modules)


class Conv2dBN2d(nn.Sequential):

    def __init__(self,
                 in_channels:  int,
                 out_channels: int,
                 kernel_size:  int = 3,
                 stride:       int = 1,
                 groups:       int = 1):

        modules = []
        modules += [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=(kernel_size - 1) // 2, groups=groups, bias=False)]
        modules += [nn.BatchNorm2d(out_channels)]

        super(Conv2dBN2d, self).__init__(*modules)


class InvertedResidual(nn.Module):

    def __init__(self,
                 in_planes:        int,
                 out_planes:       int,
                 stride:           int,
                 expansion_factor: int,
                 activation_class: Type[nn.Module] = nn.ReLU6):

        # validate `stride` argument
        if stride not in {1, 2}:
            raise ValueError

        super(InvertedResidual, self).__init__()

        hid_planes = in_planes * expansion_factor

        # build the residual branch
        layers = []
        # (optional) input layer
        if expansion_factor > 1:  # this is an "inverted" residual (instead of reducing the number of channels as in ResNets, the input layer increases it)
            # point-wise convolution
            layers += [Conv2dBN2dActivation(in_planes, hid_planes, kernel_size=1, activation_class=activation_class)]
        else:
            assert(hid_planes == in_planes)
            layers += []
        # middle layer (depth-wise convolution)
        layers += [Conv2dBN2dActivation(hid_planes, hid_planes, kernel_size=3, stride=stride, groups=hid_planes, activation_class=activation_class)]
        # output layer (point-wise convolution)
        layers += [Conv2dBN2d(hid_planes, out_planes, kernel_size=1)]
        # concatenate input, middle, and output layers
        self.residual_branch = nn.Sequential(*layers)

        # should I use skip (i.e., identity) branch?
        self._use_skip_branch = (stride == 1) and (out_planes == in_planes)

    def forward(self, x):

        if self._use_skip_branch:
            return x + self.residual_branch(x)
        else:  # this is a bottleneck layer (i.e., free from skip-branches)
            return self.residual_branch(x)


class MobileNetV2(nn.Module):

    def __init__(self,
                 config:                       str,
                 capacity:                     float = 1.0,
                 round_to_closest_multiple_of: int = 8,
                 activation:                   str = 'ReLU6',
                 n_classes:                    int = 1000,
                 seed :                        int = -1,
                 pretrained : str = None):

        # validate inputs
        config = config.upper()  # canonicalise
        if config not in CONFIGS.keys():
            raise ValueError  # invalid configuration
            
        if not (0.0 < capacity <= 1.0):
            raise ValueError  # capacity must be a positive, compressive (i.e., not greater than one) scaling factor

        activation = activation.lower()  # canonicalise
        if activation not in ACTIVATIONS:
            raise ValueError  # invalid activation function
        if activation == 'relu':
            activation_class = nn.ReLU
        else:  # activation == 'relu6':
            activation_class = nn.ReLU6

        super(MobileNetV2, self).__init__()

        # compute structural hyper-parameters of the input and output of the `features` sub-network
        out_channels_pilot    = MobileNetV2.make_divisible_by(32 * capacity, divisor=round_to_closest_multiple_of)
        in_planes_features    = out_channels_pilot
        out_planes_features   = MobileNetV2.make_divisible_by(1280 * capacity, divisor=round_to_closest_multiple_of)
        out_channels_features = out_planes_features

        # build the network
        self.pilot      = MobileNetV2.make_pilot(out_channels_pilot, activation_class)
        self.features   = MobileNetV2.make_features(config, capacity, round_to_closest_multiple_of, activation_class, in_planes_features, out_planes_features)
        self.avgpool    = MobileNetV2.make_avgpool()
        self.classifier = MobileNetV2.make_classifier(out_channels_features, n_classes)

        # initialise the parameters
        self._initialize_weights(seed)

    @staticmethod
    def make_divisible_by(dilated_n_channels: float,
                          divisor:            int,
                          min_n_channels:     Optional[int] = None) -> int:

        # canonicalise the `min_n_channels` argument
        if min_n_channels is None:
            min_n_channels = divisor
        if (min_n_channels % divisor) != 0:
            raise ValueError  # `min_n_channels` must be divisible by `divisor`

        # compute the required `n_channels`
        n_channels = max(min_n_channels, (int(dilated_n_channels + divisor / 2) // divisor) * divisor)
        if (dilated_n_channels - n_channels) / dilated_n_channels >= 0.1:
            n_channels += divisor

        return n_channels

    @staticmethod
    def make_pilot(out_channels_pilot: int,
                   activation_class:   Type[nn.Module]) -> Conv2dBNActivation:
        return Conv2dBN2dActivation(3, out_channels_pilot, kernel_size=3, stride=2, activation_class=activation_class)

    @staticmethod
    def make_features(config:                       str,
                      capacity:                     float,
                      round_to_closest_multiple_of: int,
                      activation_class:             Type[nn.Module],
                      in_planes_features:           int,
                      out_planes_features:          int) -> nn.Sequential:

        blocks = []

        # create the backbone
        in_planes = in_planes_features
        for t, c, n, s in CONFIGS[config]:
            out_planes = MobileNetV2.make_divisible_by(c * capacity, divisor=round_to_closest_multiple_of)
            for block_id in range(0, n):
                stride = s if block_id == 0 else 1
                blocks += [InvertedResidual(in_planes, out_planes, stride, t, activation_class=activation_class)]
                in_planes = out_planes

        # add a last layer
        blocks += [Conv2dBN2dActivation(in_planes, out_planes_features, kernel_size=1, activation_class=activation_class)]

        return nn.Sequential(*blocks)

    @staticmethod
    def make_avgpool() -> nn.AdaptiveAvgPool2d:
        return nn.AdaptiveAvgPool2d((1, 1))

    @staticmethod
    def make_classifier(out_channels_features: int, n_classes: int) -> nn.Sequential:

        modules = []
        modules += [nn.Dropout(0.2)]
        modules += [nn.Linear(out_channels_features, n_classes)]

        return nn.Sequential(*modules)

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        x = self.pilot(x)
        x = self.features(x)
        x = self.avgpool(x)

        x = x.view(x.size(0), -1)

        x = self.classifier(x)

        return x

    def _initialize_weights(self, seed : int = -1) -> None:

        if seed >= 0:
            torch.manual_seed(seed)

        for m in self.modules():

            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)


We are ready to create our MobileNetV2!

In [8]:
# create a MobileNetV2
config = 'standard'
capacity = 1.0
activation = 'relu6'  # other option: `relu`

mnv2 = MobileNetV2(config=config, capacity=capacity, activation=activation)


PyTorch networks are modelled as hierarchies of `Module`s.
QuantLib provides a useful `lightweight` sub-package to traverse these hierarchies and provide an overview into its atoms (i.e., non-container `Module`s).


In [9]:
import quantlib.editing.graphs as qg

mnv2_lw = qg.lw.quantlib_traverse(mnv2)
mnv2_lw.show()



                        0.pilot	Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                        1.pilot	BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                        2.pilot	ReLU6(inplace=True)
 0.0.residual_branch.0.features	Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
 1.0.residual_branch.0.features	BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 2.0.residual_branch.0.features	ReLU6(inplace=True)
 0.1.residual_branch.0.features	Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
 1.1.residual_branch.0.features	BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 0.0.residual_branch.1.features	Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
 1.0.residual_branch.1.features	BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 2.0.residual_branch.1.featur

A fundamental functionality of PyTorch is its convenient interface to use GPUs.
Before we explain how this interface works, we need to recap how PyTorch `Tensor`s work.

Each PyTorch `Tensor` is a wrapper structure around a *payload* array.
Apart from this payload, a `Tensor` has other attributes and methods, most of which operate on the payload (we call these *payload methods*).
One of these attributes is `device`, which indicates whether the payload is stored on the main memory of the computing system (i.e., the one managed directly by the CPU) or on the memory of some GPU attached to the computing system.

The C++ backend of `Tensor`s includes several implementations of payload methods, one for each computing device (CPU vs. GPU) that might be available on our computing system.
At runtime, depending on the `device` attributes of the `Tensor`s involved in a given operation, PyTorch's engine dispatches the code to the correct version of the payload method.

Most `Module`s have parameters or hyper-parameters, but moving them individually and manually to the correct device memory can become cumbersome.
Thus, PyTorch `Module`s expose a `to` method which, if passed the `device` argument, automatically moves all the parameters and hyper-parameters to the correct device memory.

If more than one GPU is available on our system, it is possible to wrap a `Module` into an `nn.DataParallel` object, which will automatically map model replicas to all the GPUs and partition mini-batches to exploit all the available computational power.
However, we note that most of QuantLib's editing functionalities only work on `Module`s.
For this reason, we prefer to keep at least one symbolic handle to the main `Module` object.


In [10]:
def maybe_migrate_to_gpu(network: nn.Module,
                         device:  torch.device,
                         n_gpus:  int) -> Tuple[nn.Module, Union[nn.Module, nn.DataParallel]]:
    """If GPUs are avaiable, migrate the network there for better time performance."""

    if n_gpus > 0:
        network = network.to(device=device)  # move the model parameters to the lead GPU
    
    if n_gpus > 1:
        maybenndp_network = nn.DataParallel(network)  # at runtime, the model will be replicated on each available GPU
    else:
        maybenndp_network = network
    
    return network, maybenndp_network


Let's map our MobileNetV2 to GPU.

In [11]:
mnv2, maybenndp_mnv2 = maybe_migrate_to_gpu(mnv2, device, n_gpus)

### Step 4: evaluate the performance of a raw network

As in most experimental sciences, also in machine learning it is crucial to perform elementary experiments.
These experiments are important to:
* validate assumptions that, if violated, could invalidate all the following experiments;
* establish baselines against which we can compare future results.

Given that in the ImageNet validation set the 1000 classes are balancedly represented, our assumption is that the accuracy of an untrained MobileNetV1 should be around 0.1%.


In [12]:
class Label(NamedTuple):
    true:      int
    predicted: int


class Evaluation(OrderedDict):

    def __setitem__(self, input_id: int, label: Label):
        if not isinstance(input_id, int):
            raise TypeError
        if not isinstance(label, Label):
            raise TypeError

        super(Evaluation, self).__setitem__(input_id, label)

    @property
    def correct(self) -> int:
        return sum((label.true == label.predicted) for label in self.values())

    @property
    def accuracy(self) -> float:
        return 100.0 * (float(self.correct) / len(self))

    def compare(self, other: Evaluation) -> float:
        """Return the percentage of matching predictions."""

        if len(set(self.keys()).symmetric_difference(set(other.keys()))) > 0:
            raise ValueError  # can only compare evaluations carried out on the same data points

        # else, I proceed with the comparison
        matched: int = 0
        for input_id, label in self.items():
            other_label = other[input_id]
            if label.predicted == other_label.predicted:
                matched += 1

        return 100.0 * (float(matched) / len(self))

    
def evaluate_network(loader:  torch.utils.data.DataLoader,
                     network: Union[nn.Module, nn.DataParallel],
                     device:  torch.device) -> Evaluation:

    if not isinstance(loader.sampler, torch.utils.data.SequentialSampler):
        raise ValueError  # the order of the data points is not deterministic, and the input IDs lose their meaning
    
    evaluation = Evaluation()
    base_input_id: int = 0

    for batch_id, (x, y_true) in enumerate(loader):
        
        x = x.to(device=device)
        y_true = y_true.to(device=device)

        y_pred = torch.argmax(network(x), dim=1)
        
        for i, (yt, yp) in enumerate(zip(y_true.flatten(), y_pred.flatten())):
            evaluation[base_input_id + i] = Label(int(yt), int(yp))
        base_input_id += len(x)
        
    return evaluation


Let's evaluate the network performance.

In [13]:
# # set the network in evaluation mode to "freeze" the parameters of batch normalisations
# maybenndp_mnv2.eval()

# # evaluate the network on the validation set
# mnv2_perf = evaluate_network(valid_loader, maybenndp_mnv2, device)
# print("Accuracy (floating-point, untrained): {:6.2f}%.".format(mnv2_perf.accuracy))

# # restore the training mode
# maybenndp_mnv2.train()
# pass


The accuracy is in line with our expecations: a positive sanity check.

## Part 2: training and evaluating a fake-quantised MobileNetV1

### Step 1: retrieve a pre-trained floating-point network

In some cases, quantisation algorithms can be applied to pre-trained floating-point networks.
These algorithms can be classified in:
* **post-training quantisation (PTQ)** algorithms, which do not need to run any gradient descent iteration or apply any parameter updates;
* **quantisation-aware fine-tuning (QAFT)** algorithms, applications of *quantisation-aware training (QAT)* algorithms that last at most a few epochs.

To speed up our work, we create a new MobileNetV2 and load such a pre-trained model.


In [14]:
# create a MobileNetV2
config = 'standard'
capacity = 1.0
activation = 'relu6'  # other option: `relu`

mnv2 = MobileNetV2(config=config, capacity=capacity, activation=activation)
mnv2, maybenndp_mnv2 = maybe_migrate_to_gpu(mnv2, device, n_gpus)

# get the path to the floating-point checkpoint
path_logs = os.path.join(os.curdir, 'logs')
fp_checkpoint_filename = '_'.join(['MNv2', str(capacity), activation]) + '.ckpt'
path_fp_checkpoint = os.path.join(path_logs, fp_checkpoint_filename)
if not os.path.isfile(path_fp_checkpoint):
    raise FileNotFoundError

# load the pre-trained parameters into the network object
pretrained_state_dict = torch.load(path_fp_checkpoint)
mnv2.load_state_dict(pretrained_state_dict)


<All keys matched successfully>

Let's evaluate the network performance.

In [15]:
# # set the network in evaluation mode to "freeze" the parameters of batch normalisations
# maybenndp_mnv2.eval()

# # evaluate the network on the validation set
# mnv2_perf = evaluate_network(valid_loader, maybenndp_mnv2, device)
# print("Accuracy (floating-point, trained): {:6.2f}%.".format(mnv2_perf.accuracy))

# # restore the training mode
# maybenndp_mnv2.train()
# pass


### Step 2: perform the float-to-fake (F2F) conversion

QuantLib's `editing` package implements the building blocks of a rudimental compiler to quantise `nn.Module`s and transform them into quantised neural networks.
This package consists of two sub-packages:
* `graphs`, extending PyTorch's `nn` and `fx` namespaces;
* `editing`, implementing the computational graph annotation and rewriting functionalities.

The first step towards the quantisation of our MobileNetV1 is replacing its composing `nn.Module`s with counterparts that support quantisation.
In QuantLib, these counterparts are `_QModule`s.

As a first step, we need to trace our floating-point network.


In [16]:
# trace
mnv2.eval()  # remember to freeze parameters, since the `Editor`s might operate with them
mnv2fp = qg.fx.quantlib_symbolic_trace(root=mnv2)


Now that we have an `fx.GraphModule` object, we can pass it to the tool performing the so-called **float-to-fake (F2F)** conversion.

In this case, we are aiming for a quantised network using signed 8-bit integers for the weight arrays, and unsigned 8-bit integers for the feature arrays.
The quantisation algorithm is **parametrised clipping activation (PACT)**.


In [17]:
import quantlib.editing.editing as qe

f2fconverter = qe.f2f.F2F8bitPACTConverter()
mnv2fq_uninit = f2fconverter(mnv2fp)


Let's visually inspect whether the conversion was successful.

In [18]:
mnv2fq_uninit_lw = qg.lw.quantlib_traverse(root=mnv2fq_uninit)
mnv2fq_uninit_lw.show()



                                                    0.pilot	PACTConv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                                                    1.pilot	BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                                                    2.pilot	PACTReLU6(inplace=True)
                             0.0.residual_branch.0.features	PACTConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                             1.0.residual_branch.0.features	BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                             2.0.residual_branch.0.features	PACTReLU6(inplace=True)
                             0.1.residual_branch.0.features	PACTConv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                             1.1.residual_branch.0.features	BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_sta

Now we are going to prepare all the ingredients to train and validate our fake-quantised network:
* the loss function; in PyTorch, loss functions are implemented as `nn.Module`s;
* the optimiser; in PyTorch, optimisers are implemented as `torch.optim.Optimizer` objects, and their responsibility is updating the parameters, **not** performing gradient descent (which is a prerogative of the *autograd engine*).


In [19]:
# if GPUs are available, migrate the network
mnv2fq_uninit, maybenndp_mnv2fq_uninit = maybe_migrate_to_gpu(mnv2fq_uninit, device, n_gpus)

# create the loss function
loss_fn = nn.CrossEntropyLoss()

# create the optimiser; since we use PACT, we need an `Optimizer` capable of updating the clipping bounds independently of other parameters
import quantlib.algorithms as qa
optimiser = qa.qalgorithms.qatalgorithms.pact.PACTSGD(mnv2fq_uninit, pact_decay=0.001, lr=0.0005, momentum=0.9, weight_decay=4e-5)


Multiple iterations through the training data set might be required to bring the model to convergence.
Since for each training iteration we also want to validate the performance of our model on the validation set, we define convenient functions to run individual training and validation epochs.


In [20]:
def train_one_epoch(loader:    torch.utils.data.DataLoader,
                    network:   Union[nn.Module, nn.DataParallel],
                    device:    torch.device,
                    loss_fn:   nn.Module,
                    optimiser: torch.optim.Optimiser,
                    verbose:   bool = False):
    
    network.train()
    
    # statistical performance counters  # TODO: define a `StatisticalPerformanceCounters` object
    n_points:   int = 0
    correct:    int = 0
    total_loss: float = 0.0
    
    for batch_id, (x, y_true) in enumerate(loader):
        
    ####################
        if 100 <= batch_id:
            break
    ####################
        
        # cast data points to the network's device
        x = x.to(device)
        y_true = y_true.to(device)

        # forward pass
        y_pred = network(x)
        loss = loss_fn(y_pred, y_true)
        
        # update performance counters
        n_points += len(x)
        correct += int(torch.sum(y_true == y_pred.argmax(dim=1)))
        total_loss = total_loss + (loss.item() * len(x))
        if verbose:
            print("Training batch [{:5d}/{:5d}] - Loss: {:8.3f} - Accuracy: {:6.2f}%".format(batch_id, len(loader), total_loss / n_points, 100.0 * (float(correct) / n_points)))
        
        # backward pass
        optimiser.zero_grad()  # clear old gradients
        loss.backward()        # compute new gradients
        optimiser.step()       # apply gradient descent step
        
        
def validate_one_epoch(loader:    torch.utils.data.DataLoader,
                       network:   Union[nn.Module, nn.DataParallel],
                       device:    torch.device,
                       loss_fn:   nn.Module,
                       verbose:   bool = False):
    
    network.eval()

    # statistical performance counters  # TODO: define a `StatisticalPerformanceCounters` object
    n_points:   int = 0
    correct:    int = 0
    total_loss: float = 0.0
        
    for batch_id, (x, y_true) in enumerate(loader):
        
        # cast data points to the network's device
        x = x.to(device)
        y_true = y_true.to(device)

        # forward pass
        y_pred = network(x)
        loss = loss_fn(y_pred, y_true)

        # update performance counters
        n_points += len(x)
        correct += int(torch.sum(y_true == y_pred.argmax(dim=1)))
        total_loss = total_loss + (loss.item() * len(x))
        if verbose:
            print("Validation batch [{:5d}/{:5d}] - Loss: {:8.3f} - Accuracy: {:6.2f}%".format(batch_id, len(loader), total_loss / n_points, 100.0 * (float(correct) / n_points)))


In [21]:
# since we run this "observation" on CPU or a single GPU, we need to limit the batch size
warmup_valid_loader = loader_factory.get_dataloader('valid', valid_transform, per_gpu_batch_size, num_workers=n_cpus)

# set validation state
mnv2fq_uninit.eval()

# collect statistics about the floating-point `Tensor`s passing through the quantisers, so that we can better fit the quantisers' hyper-parameters
# start observing
for m in mnv2fq_uninit.modules():
    if isinstance(m, tuple(qa.qalgorithms.qatalgorithms.pact.NNMODULE_TO_PACTMODULE.values())):
        m.start_observing()
# collect statistics
validate_one_epoch(warmup_valid_loader, mnv2fq_uninit, device, loss_fn)
# stop observing
for m in mnv2fq_uninit.modules():
    if isinstance(m, tuple(qa.qalgorithms.qatalgorithms.pact.NNMODULE_TO_PACTMODULE.values())):
        m.stop_observing()

# restore training state
mnv2fq_uninit.train()
        
mnv2fq_init, maybenndp_mnv2fq_init = mnv2fq_uninit, maybenndp_mnv2fq_uninit  # now the quantisers' hyper-parameters are initialised


In [22]:
# # set the network in evaluation mode to "freeze" the parameters of batch normalisations
# maybenndp_mnv2fq_init.eval()

# # evaluate the network on the validation set
# mnv2fq_perf = evaluate_network(valid_loader, maybenndp_mnv2fq_init, device)
# print("Accuracy (fake-quantised, untrained): {:6.2f}%.".format(mnv2fq_perf.accuracy))

# # restore the training mode
# maybenndp_mnv2fq_init.train()
# pass


In [23]:
fp_checkpoint_base_filename, extension = fp_checkpoint_filename.rsplit('.', 1)
fq_checkpoint_base_filename = '_'.join([fp_checkpoint_base_filename, 'FQ', 'uint8x', 'int8w'])
fq_checkpoint_filename = '.'.join([fq_checkpoint_base_filename, extension])

path_fq_checkpoint = os.path.join(path_logs, fq_checkpoint_filename)

if not os.path.isfile(path_fq_checkpoint):
    train_one_epoch(train_loader, maybenndp_mnv2fq_init, device, loss_fn, optimiser, verbose=True)
    torch.save(mnv2fq_init.state_dict(), path_fq_checkpoint)

mnv2fq_init.load_state_dict(torch.load(path_fq_checkpoint))


Training batch [    0/13346] - Loss:    1.104 - Accuracy:  67.71%
Training batch [    1/13346] - Loss:    1.046 - Accuracy:  69.79%
Training batch [    2/13346] - Loss:    0.982 - Accuracy:  71.88%
Training batch [    3/13346] - Loss:    1.016 - Accuracy:  72.14%
Training batch [    4/13346] - Loss:    1.065 - Accuracy:  72.08%
Training batch [    5/13346] - Loss:    1.069 - Accuracy:  72.57%
Training batch [    6/13346] - Loss:    1.081 - Accuracy:  72.47%
Training batch [    7/13346] - Loss:    1.056 - Accuracy:  73.05%
Training batch [    8/13346] - Loss:    1.061 - Accuracy:  72.80%
Training batch [    9/13346] - Loss:    1.064 - Accuracy:  72.71%
Training batch [   10/13346] - Loss:    1.059 - Accuracy:  73.30%
Training batch [   11/13346] - Loss:    1.053 - Accuracy:  73.61%
Training batch [   12/13346] - Loss:    1.047 - Accuracy:  73.40%
Training batch [   13/13346] - Loss:    1.060 - Accuracy:  73.21%
Training batch [   14/13346] - Loss:    1.050 - Accuracy:  73.47%
Training b

<All keys matched successfully>

In [24]:
# set the network in evaluation mode to "freeze" the parameters of batch normalisations
maybenndp_mnv2fq_init.eval()
for n, m in mnv2fq_init.named_modules():
    if isinstance(m, qg.nn.HarmonisedAdd):
        m.harmonise()

# evaluate the network on the validation set
mnv2fq_init_perf = evaluate_network(valid_loader, maybenndp_mnv2fq_init, device)
print("Accuracy (fake-quantised, trained): {:6.2f}%.".format(mnv2fq_init_perf.accuracy))

# restore the training mode
maybenndp_mnv2fq_init.train()
pass


Accuracy (fake-quantised, trained):  65.44%.


We now have a fake-quantised model with approximately the same accuracy as the floating-point one.

## Part 3: integerising a fake-quantised network

A trained fake-quantised network is not a true integerised program.
To obtain such a network, we apply the so-called **fake-to-true (F2T)** conversion, a sequence of program transformations to turn a fake-quantised network into an integerised one.


In [25]:
class MNv2Head(nn.Module):

    def __init__(self):
        super(MNv2Head, self).__init__()
        self.eps = qg.nn.EpsTunnel(torch.Tensor([1.0]))
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.drp = nn.Dropout(0.2)
        self.lin = nn.Linear(1, 1, bias=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.eps(x)
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.drp(x)
        x = self.lin(x)
        return x


class MNv2HeadApplier(qe.editors.nnmodules.NNModuleApplier):

    def __init__(self, mnv2headpattern: qe.editors.nnmodules.GenericNNModulePattern):
        super(MNv2HeadApplier, self).__init__(mnv2headpattern)

    def _apply(self, g: fx.GraphModule, ap: qe.editors.nnmodules.NodesMap, id_: str) -> fx.GraphModule:

        name_to_match_node = self.pattern.name_to_match_node(nodes_map=ap)
        node_drp = name_to_match_node['drp']
        node_lin = name_to_match_node['lin']

        name_to_match_module = self.pattern.name_to_match_module(nodes_map=ap, data_gm=g)
        module_eps = name_to_match_module['eps']
        module_lin = name_to_match_module['lin']

        assert module_eps.eps_out.numel() == 1
        assert len(node_drp.all_input_nodes) == 1
        assert len(node_lin.all_input_nodes) == 1

        # create the new module
        new_target = id_
        new_module = nn.Linear(in_features=module_lin.in_features, out_features=module_lin.out_features, bias=module_lin.bias is not None)
        new_weight = module_lin.weight.data.detach().clone() * module_eps.eps_out
        new_module.weight.data = new_weight
        if module_lin.bias is not None:
            new_bias = module_lin.bias.data.detach().clone()
            new_module.bias.data = new_bias

        # add the requantised linear operation to the graph...
        g.add_submodule(new_target, new_module)
        drp_input = next(iter(node_drp.all_input_nodes))
        with g.graph.inserting_after(drp_input):
            new_node = g.graph.call_module(new_target, args=(drp_input,))
        node_lin.replace_all_uses_with(new_node)

        module_eps.set_eps_out(torch.ones_like(module_eps.eps_out))

        # ...and delete the old operations
        g.delete_submodule(node_lin.target)
        g.graph.erase_node(node_lin)
        g.delete_submodule(node_drp.target)
        g.graph.erase_node(node_drp)

        return g


class MNv2HeadRewriter(qe.editors.nnmodules.NNModuleRewriter):

    def __init__(self):
        # create pattern
        mnv2headwithcheckers = qe.editors.nnmodules.NNModuleWithCheckers(MNv2Head(), {})
        mnv2headpattern = qe.editors.nnmodules.GenericNNModulePattern(qg.fx.quantlib_symbolic_trace, mnv2headwithcheckers)
        # create matcher and applier
        finder = qe.editors.nnmodules.GenericGraphMatcher(mnv2headpattern)
        applier = MNv2HeadApplier(mnv2headpattern)
        # link pattern, matcher, and applier into the rewriter
        super(MNv2HeadRewriter, self).__init__('MNv2HeadRewriter', mnv2headpattern, finder, applier)


In [26]:
# F2T conversion and ONNX exporting require structural information about the input
x, _ = next(iter(valid_loader))
x = x[0].unsqueeze(0)

# set the network in evaluation mode to "freeze" the parameters of batch normalisations
mnv2fq_init = mnv2fq_init.to(torch.device('cpu'))  # TODO: the `Tensor`s generated inside the F2T conversion flow are generated for CPU
mnv2fq_init.eval()

# perform the conversion
f2tconverter = qe.f2t.F2T24bitConverter(custom_editor=MNv2HeadRewriter())
mnv2tq = f2tconverter(mnv2fq_init, {'x': {'shape': x.shape, 'scale': torch.Tensor([ImageNetStats['quantise']['scale']])}})


/home/spmatteo/anaconda3/envs/quantlab/lib/python3.8/site-packages/quantlib-0.2-py3.8.egg/quantlib/editing/editing/editors/nnmodules/finder/nnsequential.py:215: UserWarning: [QuantLib warning] RequantiserMatcher: two matches with non-zero overlap were found; the most recently discovered will be discarded.
  warnings.warn(quantlib_wng_header(obj_name=self.__class__.__name__) + "two matches with non-zero overlap were found; the most recently discovered will be discarded.")


To validate the performance of the integerised network, we need to pass it integerised data.
Thus, we create a `DataLoader` yielding `Tensor` images with integer components.


In [27]:
# create the validation `DataLoader` returning integerised (UINT8) images
int_valid_transform = ImageNetTransform('valid', integerise=True)
int_valid_loader = loader_factory.get_dataloader('valid', int_valid_transform, per_gpu_batch_size, num_workers=n_cpus)


Let's evaluate our true-quantised MobileNetV2.

In [28]:
mnv2tq = mnv2tq.to(device=device)

# set the network in evaluation mode to "freeze" the parameters of batch normalisations
mnv2tq.eval()

# evaluate the network on the validation set
mnv2tq_perf = evaluate_network(int_valid_loader, mnv2tq, device)
print("Accuracy (true-quantised, trained): {:6.2f}%.".format(mnv2tq_perf.accuracy))


Accuracy (true-quantised, trained):  65.80%.


## Part 4: export a backend-specific ONNX model

The QuantLib `backends` package contains the abstractions required to export **ONNX** models, and possibly annotate them with backend-specific information.
To demonstrate its usage, we consider the [DORY](https://github.com/pulp-platform/dory) backend.

First, we create the folder to host DORY-specific files.


In [29]:
import shutil

backend_name = 'DORY'
path_export = os.path.join(os.curdir, backend_name)

if os.path.isdir(path_export):  # remove old files
    shutil.rmtree(path_export)
os.mkdir(path_export)


Then, we export an annotated ONNX model.
DORY can also verify whether the exported ONNX model can be correctly transformed into an integerised program for PULP platforms.
To perform this consistency checks, we need to dump the input and features maps corresponding to an example data point.


In [30]:
import quantlib.backends as qb

x = x.to(torch.device('cpu'))
mnv2tq = mnv2tq.to(device=torch.device('cpu'))

exporter = qb.dory.DORYExporter()
exporter.export(network=mnv2tq, input_shape=x.shape, path=path_export)
exporter.dump_features(network=mnv2tq, x=x, path=path_export)


We can use [Netron](https://netron.app/) to visually inspect that the exported model is an integerised program.

## Conclusion

We have reached the bottom of the part of the deep learning stack covered by QuantLib.
If you are interested in graph optimisations and code generation, you can read the [DORY paper](https://ieeexplore.ieee.org/document/9381618) and check out the DORY repository.

Cheers!
